# Sentiment Classification


### Generate Word Embeddings and retrieve outputs of each layer with Keras based on Classification task

Word embeddings are a type of word representation that allows words with similar meaning to have a similar representation.

It is a distributed representation for text that is perhaps one of the key breakthroughs for the impressive performance of deep learning methods on challenging natural language processing problems.

We willl use the imdb dataset to learn word embeddings as we train our dataset. This dataset contains 25,000 movie reviews from IMDB, labeled with sentiment (positive or negative). 



### Dataset

`from keras.datasets import imdb`

Dataset of 25,000 movies reviews from IMDB, labeled by sentiment (positive/negative). Reviews have been preprocessed, and each review is encoded as a sequence of word indexes (integers). For convenience, the words are indexed by their frequency in the dataset, meaning the for that has index 1 is the most frequent word. Use the first 20 words from each review to speed up training, using a max vocab size of 10,000.

As a convention, "0" does not stand for a specific word, but instead is used to encode any unknown word.


### Aim

1. Import test and train data  
2. Import the labels ( train and test) 
3. Get the word index and then Create key value pair for word and word_id. (12.5 points)
4. Build a Sequential Model using Keras for Sentiment Classification task. (10 points)
5. Report the Accuracy of the model. (5 points)  
6. Retrive the output of each layer in keras for a given single test sample from the trained model you built. (2.5 points)


#### Usage:

In [0]:
from keras.datasets import imdb
import numpy as np
# save np.load
np_load_old = np.load

# modify the default parameters of np.load
np.load = lambda *a,**k: np_load_old(*a, allow_pickle=True, **k)

vocab_size = 10000 #vocab size

(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=vocab_size) # vocab_size is no.of words to consider from the dataset, ordering based on frequency.

# restore np.load for future normal usage
np.load = np_load_old

In [55]:
x_train.shape

(25000,)

In [56]:
x_test.shape

(25000,)

In [0]:
from keras.preprocessing.sequence import pad_sequences
vocab_size = 10000 #vocab size
maxlen = 300  #number of word used from each review

In [0]:
#load dataset as a list of ints
#(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=vocab_size)
#make all sequences of the same length
x_train = pad_sequences(x_train, maxlen=maxlen)
x_test =  pad_sequences(x_test, maxlen=maxlen)

In [59]:
print(x_train)

[[   0    0    0 ...   19  178   32]
 [   0    0    0 ...   16  145   95]
 [   0    0    0 ...    7  129  113]
 ...
 [   0    0    0 ...    4 3586    2]
 [   0    0    0 ...   12    9   23]
 [   0    0    0 ...  204  131    9]]


In [60]:
print(x_test)

[[   0    0    0 ...   14    6  717]
 [   0    0    0 ...  125    4 3077]
 [1239 5189  137 ...    9   57  975]
 ...
 [   0    0    0 ...   21  846 5518]
 [   0    0    0 ... 2302    7  470]
 [   0    0    0 ...   34 2005 2643]]


*italicized text*## Build Keras Embedding Layer Model
We can think of the Embedding layer as a dicionary that maps a index assigned to a word to a word vector. This layer is very flexible and can be used in a few ways:

* The embedding layer can be used at the start of a larger deep learning model. 
* Also we could load pre-train word embeddings into the embedding layer when we create our model.
* Use the embedding layer to train our own word2vec models.

The keras embedding layer doesn't require us to onehot encode our words, instead we have to give each word a unqiue intger number as an id. For the imdb dataset we've loaded this has already been done, but if this wasn't the case we could use sklearn [LabelEncoder](http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.LabelEncoder.html).

In [0]:
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential, load_model
from keras.layers import LSTM, Dense, Dropout, Embedding, Masking, Bidirectional,SimpleRNN
from keras.optimizers import Adam
from keras import backend

In [62]:
x_train.shape

(25000, 300)

In [63]:
x_test.shape

(25000, 300)

In [64]:
model = Sequential()
model.add(Embedding(vocab_size, 8, input_length=maxlen))
model.add(
    SimpleRNN(
        1, return_sequences=False, dropout=0.1))
model.add(Dense(1, activation='sigmoid'))
# compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
# summarize the model
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_11 (Embedding)     (None, 300, 8)            80000     
_________________________________________________________________
simple_rnn_4 (SimpleRNN)     (None, 1)                 10        
_________________________________________________________________
dense_16 (Dense)             (None, 1)                 2         
Total params: 80,012
Trainable params: 80,012
Non-trainable params: 0
_________________________________________________________________
None


## Retrive the output of each layer in keras for a given single test sample from the trained model you built

In [65]:
# fit the model
model.fit(x_train, y_train, epochs=50)

Epoch 1/50
25000/25000 [==============================] - 172s 7ms/step - loss: 0.5547 - acc: 0.7504
Epoch 2/50
25000/25000 [==============================] - 171s 7ms/step - loss: 0.4509 - acc: 0.8154
Epoch 3/50
25000/25000 [==============================] - 171s 7ms/step - loss: 0.3945 - acc: 0.8456
Epoch 4/50
25000/25000 [==============================] - 171s 7ms/step - loss: 0.3795 - acc: 0.8498
Epoch 5/50
25000/25000 [==============================] - 170s 7ms/step - loss: 0.3620 - acc: 0.8596
Epoch 6/50
25000/25000 [==============================] - 170s 7ms/step - loss: 0.3561 - acc: 0.8605
Epoch 7/50
25000/25000 [==============================] - 170s 7ms/step - loss: 0.3759 - acc: 0.8482
Epoch 8/50
25000/25000 [==============================] - 171s 7ms/step - loss: 0.3345 - acc: 0.8712
Epoch 9/50
25000/25000 [==============================] - 172s 7ms/step - loss: 0.3161 - acc: 0.8814
Epoch 10/50
25000/25000 [==============================] - 171s 7ms/step - loss: 0.3254 - a

In [0]:
model.save("./SeqNLP_Project1.h5") #Save the long running model

In [71]:
loss, accuracy = model.evaluate(x_test, y_test)
print('Accuracy: %f' % (accuracy*100))
print('Loss: %f' % (loss))

25000/25000 [==============================] - 94s 4ms/step
Accuracy: 76.448000
Loss: 0.681476


In [72]:
model.predict(x_test[[0]])

array([[0.877219]], dtype=float32)

In [74]:
from keras import backend as back

inpt = model.input
output = [layer.output for layer in model.layers]
evalFunction = [back.function([inpt, back.learning_phase()], [out]) for out in output]

layerOpt = [func([x_test[[0]], 1.]) for func in evalFunction]
print(layerOpt)


[[array([[[ 0.02326414,  0.02180665, -0.0241294 , ...,  0.00594948,
         -0.01203327, -0.01301493],
        [ 0.02326414,  0.02180665, -0.0241294 , ...,  0.00594948,
         -0.01203327, -0.01301493],
        [ 0.02326414,  0.02180665, -0.0241294 , ...,  0.00594948,
         -0.01203327, -0.01301493],
        ...,
        [-0.02587913, -0.11960392, -0.04466499, ...,  0.02193185,
         -0.00487106,  0.11468413],
        [-0.0120837 ,  0.05895125,  0.01856722, ..., -0.02211373,
          0.02207558, -0.06055649],
        [ 0.04184148,  0.09781101,  0.11669611, ..., -0.05276844,
          0.11992112, -0.10343277]]], dtype=float32)], [array([[-0.2999501]], dtype=float32)], [array([[0.92966336]], dtype=float32)]]


In [78]:
print('Embedding layer Output')
layerOpt[0][0]

Embedding layer Output


array([[[ 0.02326414,  0.02180665, -0.0241294 , ...,  0.00594948,
         -0.01203327, -0.01301493],
        [ 0.02326414,  0.02180665, -0.0241294 , ...,  0.00594948,
         -0.01203327, -0.01301493],
        [ 0.02326414,  0.02180665, -0.0241294 , ...,  0.00594948,
         -0.01203327, -0.01301493],
        ...,
        [-0.02587913, -0.11960392, -0.04466499, ...,  0.02193185,
         -0.00487106,  0.11468413],
        [-0.0120837 ,  0.05895125,  0.01856722, ..., -0.02211373,
          0.02207558, -0.06055649],
        [ 0.04184148,  0.09781101,  0.11669611, ..., -0.05276844,
          0.11992112, -0.10343277]]], dtype=float32)

In [79]:
print('RNN Output')
layerOpt[1]

RNN Output


[array([[-0.2999501]], dtype=float32)]

In [80]:
print('Dense Layer output')
layerOpt[2]

Dense Layer output


[array([[0.92966336]], dtype=float32)]